In [12]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
import numpy as np

# 1. Charger les données
df = pd.read_csv("C:/Users/HP G3/Bureau/voitures_preprocessed.csv", encoding='latin1')

# 2. Supprimer les lignes avec trop de valeurs manquantes
seuil_null = df.shape[1] * 0.5
df = df[df.isnull().sum(axis=1) < seuil_null]

# 3. Nettoyer les années et calculer l'âge
df['Année'] = pd.to_numeric(df['Année'], errors='coerce').fillna(1980).astype(int)
df['Age'] = 2025 - df['Année']

# 4. Utiliser la colonne 'Prix' telle quelle mais appliquer une transformation logarithmique pour la visualisation
df['Log_Prix'] = np.log1p(df['Prix'])  # log1p est équivalent à log(Prix + 1) pour éviter les problèmes avec les valeurs nulles ou zéro

# 5. Fusionner les colonnes carburant, boîte à vitesses et origine
# Fusionner les types de carburant
carburant_cols = ['Carburant_Diesel', 'Carburant_Electrique', 'Carburant_Essence', 'Carburant_Hybride', 'Carburant_LPG']
df['Carburant'] = df[carburant_cols].idxmax(axis=1).str.replace("Carburant_", "")

# Fusionner les types de boîte à vitesses
boitev_cols = ['BoiteàV_Automatique', 'BoiteàV_Manuelle']
df['BoiteàV'] = df[boitev_cols].idxmax(axis=1).str.replace("BoiteàV_", "")

# Fusionner les types d'origine
origine_cols = ['Origine_Dédouanée', 'Origine_Importée_neuve', 'Origine_Pas_encore_dédouanée', 'Origine_WW_au_Maroc']
df['Origine'] = df[origine_cols].idxmax(axis=1).str.replace("Origine_", "")

# 6. Heatmap des corrélations avec les colonnes fusionnées
df_corr = df.copy()

# Conversion des colonnes catégorielles (Carburant, Origine, Boite à vitesses) en numériques
df_corr['Carburant'] = df_corr['Carburant'].astype('category').cat.codes
df_corr['Origine'] = df_corr['Origine'].astype('category').cat.codes
df_corr['BoiteàV'] = df_corr['BoiteàV'].astype('category').cat.codes

# Sélectionner les colonnes pertinentes pour la corrélation
df_corr = df_corr[['Prix', 'Log_Prix', 'Age', 'Carburant', 'BoiteàV', 'Origine']]

# Calculer la matrice de corrélation
corr_matrix = df_corr.corr()

# 7. Heatmap avec une taille plus modérée
fig_heatmap = px.imshow(corr_matrix, text_auto=True, aspect="auto", width=800, height=600,  # Taille réduite
                        title="Heatmap des Corrélations ",
                        color_continuous_scale="YlGnBu")

# 8. Histogramme de la distribution des prix avec transformation logarithmique
fig_price = px.histogram(df, x='Log_Prix', nbins=30, title='Distribution des Prix',
                         labels={'Log_Prix': 'Log(Prix + 1)'},
                         template="plotly_dark", color_discrete_sequence=["#FF6347"],
                         log_y=True)  # Échelle logarithmique

# 9. Répartition des prix par marque-modèle
fig_boxplot = px.histogram(df, x='Marque_Modele', y='Log_Prix', title="Prix par Marque et Modèle",
                           template="plotly_dark", color_discrete_sequence=["#8A2BE2"], log_y=True)

# 10. Prix par type de carburant
fig_carburant = px.histogram(df, x='Carburant', y='Log_Prix', title="Prix par Type de Carburant",
                             template="plotly_dark", color_discrete_sequence=["#3CB371"], log_y=True)

# 11. Impact de l'âge sur le prix
fig_age = px.histogram(df, x='Age', y='Log_Prix', title="Impact de l'Âge sur le Prix",
                       template="plotly_dark", color_discrete_sequence=["#FF1493"], log_y=True)

# 12. Top 10 modèles les plus fréquents
top_models = df['Marque_Modele'].value_counts().head(10)
fig_models = px.bar(x=top_models.index, y=top_models.values,
                    title="Top 10 des Modèles les Plus Populaires",
                    labels={'x': 'Modèle', 'y': 'Occurrences'}, template="plotly_dark",
                    color_discrete_sequence=["#FFD700"])

# 13. Répartition par Origine
fig_origine = px.pie(df, names='Origine', title="Répartition par Origine",
                     template="plotly_dark", color_discrete_sequence=["#FF8C00", "#DAA520", "#CD5C5C", "#6A5ACD"])

# 14. Répartition par Boîte à vitesses
fig_boite_vitesse = px.pie(df, names='BoiteàV', title="Répartition par Type de Boîte à Vitesses",
                           template="plotly_dark", color_discrete_sequence=["#FF6347", "#4682B4"])

# 15. Lancer Dash
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1("Analyse Exploratoire - Marché Automobile Marocain", style={'textAlign': 'center', 'color': '#FF6347'}),
    html.Div([
        dcc.Graph(figure=fig_price),
        dcc.Graph(figure=fig_boxplot),
        dcc.Graph(figure=fig_carburant),
        dcc.Graph(figure=fig_age),
        dcc.Graph(figure=fig_heatmap),
        dcc.Graph(figure=fig_models),
        dcc.Graph(figure=fig_origine),
        dcc.Graph(figure=fig_boite_vitesse)
    ], style={'display': 'grid', 'gridTemplateColumns': 'repeat(2, 2fr)', 'gap': '10px'})
])

if __name__ == '__main__':
    app.run(debug=True, port=8063) 

In [2]:
import pandas as pd
import numpy as np


# Chargement des données
df = pd.read_csv("C:/Users/HP G3/Bureau/voitures_preprocessed.csv", encoding='latin1')

print(df["Année"].unique())

['2019' '2020' '2024' '2012' '2016' '2021' '2022' '2023' '2017' '2011'
 '2014' '2006' '2010' '2013' '1999' '2007' '2018' '1993' '2015' '2004'
 '2008' '2009' '2002' '2001' '2003' '1991' '2025' '1982'
 '1980 ou plus ancien' '2000' '1995' '1998' '2005' '1984' '1990' '1988'
 '1985' '1994' '1996' '1997' '1986' '1983' '1987' '1992' '1989' '1981']
